# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [6]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

In [45]:
df = pd.read_csv('homepage_actions.csv')
df.shape

(8188, 4)

In [4]:
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [20]:
view = df.loc[df['action'] == 'view']
click = df.loc[df['action'] == 'click']

print('views without clicking: {}'.format( len(set(view['id'].to_list()) - set(click['id'].to_list())) ))
print('clicks without viewing: {}'.format( len(set(click['id'].to_list()) - set(view['id'].to_list())) ))

views without clicking: 4468
clicks without viewing: 0


In [23]:
experiment = df.loc[df['group'] == 'experiment']
control = df.loc[df['group'] == 'control']

print('users in both experiment and control: {}'\
      .format( len(set(experiment['id'].to_list()) & set(control['id'].to_list())) ))

users in both experiment and control: 0


In [3]:
"""
1860 people viewed then clicked, no anomalies (people who clicked without viewing, viewed or clicked more than once)
"""

'\n1860 people viewed then clicked, no anomalies (people who clicked without viewing, viewed or clicked more than once)\n'

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [55]:
"""
H_null : experimental homepage and control homepage have identical click rates
H_alt  : experimental homepage has higher click rate than control
"""
df['val'] = 1
control = df.loc[df['group'] == 'control'].pivot(index = 'id',columns='action',values='val').fillna(0)
experiment = df.loc[df['group'] == 'experiment'].pivot(index = 'id',columns='action',values='val').fillna(0)

stats.ttest_ind(control.click, experiment.click, alternative='less')

Ttest_indResult(statistic=-2.6195696844542202, pvalue=0.008825098914958293)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [64]:
#Your code here
n = experiment.view.sum()
p = control.click.mean()
experiment_expected_clicks = n*p 
experiment_expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [65]:
#Your code here
stdev = np.sqrt(n*p*(1-p))
z = (experiment.click.sum() - experiment_expected_clicks) / stdev
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [66]:
#Your code here
stats.norm.sf(z)

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Yes, while the p-value is slightly lower, 
both would lead to confident rejection of the null hypothesis. 
The experimental page appears to be a more effective design.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.